**Long Term Stock Price growth prediction using NLP on 10-
K Financial Reports** 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#Change directory to MyDrive
cd /content/gdrive/My Drive


/content/gdrive/My Drive


In [3]:
import numpy as np
import pandas as pd
import os

import re
import requests

!pip install urllib3
import urllib.request as url 

!pip install bs4
from bs4 import BeautifulSoup as bs

!pip install wordcloud
from wordcloud import WordCloud

!pip install -U spacy==2.1.8
import spacy

import matplotlib.pyplot as plt
%matplotlib inline

import random
random.seed(123)

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

Requirement already up-to-date: spacy==2.1.8 in /usr/local/lib/python3.6/dist-packages (2.1.8)


In [0]:
import spacy 
nlp = spacy.load("en_core_web_sm")
from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
print(spacy.__version__)

2.1.8


In [0]:
#Read the file
fin_reports1 = pd.read_csv('./Input/10k_filing_info_train_links-1570100710859.csv',header=0, sep=',')


In [0]:
fin_reports1.shape

(2568, 5)

In [0]:
# I have done preprocessing, trained the models taking only 1000 rows from the input file(Train data)
# The reason is when I try to pre-process taking all 2568 rows, my session crashes in google colab as entire 25GB of RAM gets used.
# So all my processing, predictions submitted is on training 1000 links only. After PHD, my aim would be to work on the project taking complete input.

fin_reports2 = fin_reports1.head(1000)

In [0]:
#Dimension of the dataframe
fin_reports2.shape
    

(1000, 5)

In [0]:
fin_reports2.columns

Index(['ticker', 'cik', 'filing_date', '10k_link', 'long_term_outlook'], dtype='object')

**Web scrapping and parsing**

In [0]:
# I have done web scrapping using the requests.get(link) function. 
# From the scrapped data, searched for <TYPE>10-K report using regular expression <TYPE>10-K.*?</TEXT>
# Used Beautiful soup library for HTML parsing, to efficiently extract the text values from the HTML document.
# The challenges that I faced in web scrapping was 
# 1. Finding the right regular expression
# 2. Scraping large amount of data

%%time
fin_reports2['filename']=[]
fin_reports3 = {'ticker':[], 'cik':[], 'filing_date':[], '10k_link':[],'long_term_outlook':[], 'report_data':[]}
for row_num, row in fin_reports2.iterrows():
    ticker = row['ticker']
    cik = row['cik']
    filing_date = row['filing_date']
    tenk_link = row['10k_link']
    long_term_outlook = row['long_term_outlook']
    
    print(row_num)
    print(fin_reports2.iloc[row_num, 1],fin_reports2.iloc[row_num, 2],fin_reports2.iloc[row_num, 3])
    link = fin_reports2.iloc[row_num, 3]
    response = requests.get(link)
    text_data = response.text
    matches = re.search("(?s)<TYPE>10-K.*?</TEXT>", text_data, re.M)
    
    soup = bs(matches.group(), 'html.parser')
    file = open(os.path.join("./10k_reports/", "report_" + str(row_num) + ".htm"),"w")
    file.write(soup.get_text())
    fin_reports2['filename'][row_num] = os.path.basename(file.name)
    
    fin_reports3['ticker'].append(ticker)
    fin_reports3['cik'].append(cik)
    fin_reports3['filing_date'].append(filing_date)
    fin_reports3['10k_link'].append(tenk_link)
    fin_reports3['long_term_outlook'].append(long_term_outlook)
    fin_reports3['report_data'].append(soup.get_text())

0
1031296 2012-02-28 https://www.sec.gov/Archives/edgar/data/1031296/000103129612000012/0001031296-12-000012.txt
1
21665 2015-02-19 https://www.sec.gov/Archives/edgar/data/21665/000162828015000846/0001628280-15-000846.txt
2
1137774 2016-02-19 https://www.sec.gov/Archives/edgar/data/1137774/000113777416000226/0001137774-16-000226.txt
3
1065088 2001-03-28 https://www.sec.gov/Archives/edgar/data/1065088/000109581101001836/0001095811-01-001836.txt
4
941548 2005-02-25 https://www.sec.gov/Archives/edgar/data/941548/000095012905001646/0000950129-05-001646.txt
5
77360 2015-02-24 https://www.sec.gov/Archives/edgar/data/77360/000007736015000007/0000077360-15-000007.txt
6
21665 1997-03-18 https://www.sec.gov/Archives/edgar/data/21665/0000940180-97-000241.txt
7
811156 2002-03-29 https://www.sec.gov/Archives/edgar/data/811156/000095012402001143/0000950124-02-001143.txt
8
12659 2008-06-30 https://www.sec.gov/Archives/edgar/data/12659/000095013408012051/0000950134-08-012051.txt
9
320335 2000-03-23 ht

In [0]:
# I extract only the year from the filing year. I preassume the financial year would have an impact on the stock price prediction,
# like for example financial crisis in the year 2008
filing_year=[]
for date in fin_reports3['filing_date']:
   
    extract_year = slice(4)
    year = date[extract_year]
    #print(year)
    filing_year.append(year)

fin_reports3['filing_year'] = filing_year

In [0]:
#Understanding the data
pd.DataFrame(fin_reports3).head()


,ticker,cik,filing_date,10k_link,long_term_outlook,report_data,filing_year
0,FE,1031296,2012-02-28,https://www.sec.gov/Archives/edgar/data/103129...,0,10-K\n1\nfe-12312011x10k.htm\n10-K\n\n\n\n\n\n...,2012
1,CL,21665,2015-02-19,https://www.sec.gov/Archives/edgar/data/21665/...,0,10-K\n1\ncl-12312014x10k.htm\nCOLGATE-PALMOLIV...,2015
2,PRU,1137774,2016-02-19,https://www.sec.gov/Archives/edgar/data/113777...,1,10-K\n1\npfi201510-k.htm\n10-K\n\n\n\n\n\n\n10...,2016
3,EBAY,1065088,2001-03-28,https://www.sec.gov/Archives/edgar/data/106508...,1,10-K\n1\nf70837e10-k.txt\nFORM 10-K FISCAL YEA...,2001
4,CAM,941548,2005-02-25,https://www.sec.gov/Archives/edgar/data/941548...,1,10-K\n1\nh21764e10vk.htm\nCOOPER CAMERON CORP....,2005


**Saving the webscraped data to text file**




In [0]:
pd.DataFrame(fin_reports3).to_csv("Extracted_train_reports_1000.csv",sep=',', header=True, index=False)
                  

In [6]:
fin_reports3 = pd.read_csv("Extracted_train_reports_1000.csv", sep= ',')
fin_reports3.head(2)
type(fin_reports3)

pandas.core.frame.DataFrame

**Text Preprocessing Steps**

**LowerCase all text - to maintain uniformity across all words and for comparison purpose**

In [0]:
fin_reports3['report_data'] = [text.strip().lower() for text in fin_reports3['report_data']]
#fin_reports3['report_data'][:2]

**Remove accented characters**

In [0]:
import unicodedata
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #https://docs.python.org/2/library/unicodedata.html
    return text

In [0]:
fin_reports3['report_data'] = [remove_accented_chars(text) for text in fin_reports3['report_data']]
#fin_reports3['report_data'][0]

**Remove scrub words**

In [0]:
def scrub_words(text):
    #Replace \xao characters in text
    text = re.sub('\xa0', ' ', text)
    
    #Replace non ascii / not words and digits
    text = re.sub("(\\W|\\d)",' ',text)
    
    #Replace new line characters and following text untill space
    text = re.sub('\n(\w*?)[\s]', '', text)
    
    #Remove html markup
    text = re.sub("<.*?>", ' ', text)
    
    #Remove extra spaces from the text
    text = re.sub("\s+", ' ', text)
    return text

In [0]:
fin_reports3['report_data'] = [scrub_words(text) for text in fin_reports3['report_data']]
#fin_reports3['report_data'][0]

**Adding new column "word_count" which specifies the number of tokens in each document**

In [0]:
fin_reports3['word_count'] = [len(text.split(' ')) for text in fin_reports3['report_data']]
pd.DataFrame(fin_reports3['word_count']).describe()

,word_count
count,2.568000e+03
mean,7.172929e+04
std,6.364039e+04
min,4.074000e+03
25%,3.718075e+04
50%,5.967550e+04
75%,8.734400e+04
max,1.308419e+06


**Removing Stopwords, tokenizing and lemmatization**

In [0]:
## load spacy's English stopwords as variable called 'stopwords'

stopwords = spacy.lang.en.stop_words.STOP_WORDS
print('Number of stop words: %d' % len(stopwords))
print('First ten stop words: %s' % list(stopwords)[:10])


Number of stop words: 326
First ten stop words: ['‘ve', 'still', 'thereafter', 'beyond', 'too', 'call', 'its', 'per', 'who', 'else']


In [0]:
# I have added the below stopwords after seeing through many of the report data
customize_stop_words = ['fe', 'htm','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','ii', 'iii', 'x','cl','aaa','aaaa','ww','yy','yyy','zz','zzz']

for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

In [0]:
def tokenize_and_lemmatize(doc, remove_stopwords = True):
    
    if remove_stopwords:
        tokens = [word for word in doc if not word.is_stop]
    else:
        tokens = [word for word in doc]
        
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token.text):
            filtered_tokens.append(token)
            
    lemma = [t.lemma_ for t in filtered_tokens]
    return lemma

In [0]:
# Tokenization and lemmatization consumed the maximum time and memory of all the steps done. 

%%time
clean_text_lemmatized = []
nlp.max_length = 40000000
for idx,text in enumerate(fin_reports3['report_data']):

    doc = nlp(text)
    print(f"processing {idx} document")
    words_lemmatized = tokenize_and_lemmatize(doc)
    clean_text_lemmatized.append(words_lemmatized)
    
fin_reports3['clean_text_lemmatized'] = clean_text_lemmatized
fin_reports3['clean_count'] = fin_reports3['clean_text_lemmatized'].apply(len)


processing 0 document
processing 1 document
processing 2 document
processing 3 document
processing 4 document
processing 5 document
processing 6 document
processing 7 document
processing 8 document
processing 9 document
processing 10 document
processing 11 document
processing 12 document
processing 13 document
processing 14 document
processing 15 document
processing 16 document
processing 17 document
processing 18 document
processing 19 document
processing 20 document
processing 21 document
processing 22 document
processing 23 document
processing 24 document
processing 25 document
processing 26 document
processing 27 document
processing 28 document
processing 29 document
processing 30 document
processing 31 document
processing 32 document
processing 33 document
processing 34 document
processing 35 document
processing 36 document
processing 37 document
processing 38 document
processing 39 document
processing 40 document
processing 41 document
processing 42 document
processing 43 documen

**Finding unique words in the list of lemmatized words**

In [0]:
unique_list = []
for row_num in range(len(fin_reports3)):
     list_set = set(fin_reports3['clean_text_lemmatized'][row_num])
     unique_list.append(list(list_set))
     
fin_reports3['clean_unique_words'] = unique_list

In [0]:
fin_reports3['clean_unique_word_count'] = fin_reports3['clean_unique_words'].apply(len)

In [0]:
fin_reports3.head(2)

In [0]:
fin_reports3.to_csv('pre-processed_train_reports_1000.',index=False)

In [0]:
fin_reports3 = pd.read_csv('pre-processed_train_reports_1000.csv',sep=',')

**Visualization - Finding most frequent words using WordCloud**

In [0]:
import nltk
from nltk.tokenize import word_tokenize

for row_num in range(len(fin_reports3)):
    frequency_dist = nltk.FreqDist(fin_reports3['clean_text_lemmatized'][row_num])
    sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:50]
    
    wordcloud = WordCloud().generate_from_frequencies(frequency_dist)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [0]:
# I have combined all the clean tokenized words to a single string, in order to pass to tf-idf vectorizer
report_string_list =[]
for row_num in range(len(fin_reports3)):
  report_string =""
  report_string = " ".join(fin_reports3['clean_text_lemmatized'][row_num])
  report_string_list.append(report_string)

In [0]:
fin_reports3['report_string_list'] = report_string_list

In [0]:
fin_reports3.head(2)

,ticker,cik,filing_date,10k_link,long_term_outlook,report_data,filing_year,word_count,clean_text_lemmatized,clean_count,clean_unique_words,clean_unique_word_count,report_string_list
0,FE,1031296,2012-02-28,https://www.sec.gov/Archives/edgar/data/103129...,0,k fe x k htm k fe k united states securities ...,2012,161216,"[united, state, security, exchange, commission...",99649,"[scrubber, solid, llpcleveland, inquiry, prepa...",3871,united state security exchange commissionwashi...
1,CL,21665,2015-02-19,https://www.sec.gov/Archives/edgar/data/21665/...,0,k cl x k htm colgate palmolive k cl k united ...,2015,44390,"[colgate, palmolive, united, state, security, ...",26447,"[solid, laundry, inquiry, prepare, indicate, u...",2215,colgate palmolive united state security exchan...


In [0]:
# Calculated count of all words using count vectorizer but didnot use it anywhere further.
cv=CountVectorizer(stop_words='english',lowercase=True, decode_error='ignore')

tdm = cv.fit_transform(report_string_list)
type(tdm)

scipy.sparse.csr.csr_matrix

In [0]:
Mat = tdm.todense()
type(Mat)

numpy.matrix

In [0]:
Mat.shape

(1000, 88071)

In [0]:
Mat = pd.DataFrame(Mat)
Mat.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88031,88032,88033,88034,88035,88036,88037,88038,88039,88040,88041,88042,88043,88044,88045,88046,88047,88048,88049,88050,88051,88052,88053,88054,88055,88056,88057,88058,88059,88060,88061,88062,88063,88064,88065,88066,88067,88068,88069,88070
0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Using the finance word dictionary**

In [0]:
# Read the csv file
finance_word_dict = pd.read_csv('./Input/finance_word_dictionary-1570171303593.csv',header=0, sep=',')

In [0]:
finance_word_dict.shape

(85221, 19)

In [0]:
finance_word_dict.head()

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,275,1.603442e-08,1.306189e-08,3.665256e-06,82,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.749209e-10,1.028197e-11,1.014208e-08,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,4.664558e-10,1.465871e-10,6.401309e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,6,3.498419e-10,1.758203e-10,7.213526e-08,6,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,6729,3.923477e-07,3.752169e-07,3.452425e-05,845,0,0,0,0,0,0,0,0,0,0,3,12of12inf


In [0]:
# There are 2355 positive words in the dictionary
finance_word_dict.Negative.value_counts()

0    82866
1     2355
Name: Negative, dtype: int64

In [0]:
# There are 354 positive words in the dictionary
finance_word_dict.Positive.value_counts()

0    84867
1      354
Name: Positive, dtype: int64

In [0]:
finance_word_dict.columns

Index(['Word', 'Sequence Number', 'Word Count', 'Word Proportion',
       'Average Proportion', 'Std Dev', 'Doc Count', 'Negative', 'Positive',
       'Uncertainty', 'Litigious', 'Constraining', 'Superfluous',
       'Interesting', 'Modal', 'Irr_Verb', 'Harvard_IV', 'Syllables',
       'Source'],
      dtype='object')

In [0]:
finance_word_dict.dtypes

Word                   object
Sequence Number         int64
Word Count              int64
Word Proportion       float64
Average Proportion    float64
Std Dev               float64
Doc Count               int64
Negative                int64
Positive                int64
Uncertainty             int64
Litigious               int64
Constraining            int64
Superfluous             int64
Interesting             int64
Modal                   int64
Irr_Verb                int64
Harvard_IV              int64
Syllables               int64
Source                 object
dtype: object

In [0]:
# Converting all the words in the dictionary to lower case
finance_word_dict['Word'] = finance_word_dict['Word'].str.lower()

In [0]:
# Finding the value counts of each numeric column in the dictionary
for col in finance_word_dict.columns:
    if finance_word_dict[col].dtype == 'int64' or finance_word_dict[col].dtype == 'float64':
        print("Column = ", col, "\n")
        print(finance_word_dict[col].value_counts())
        print("\n--------------------------------- \n")

**Finding count of positive, negative, uncertain, litigious, constraining, interesting and superfluous words in each document**

In [0]:
is_Negative =  finance_word_dict['Negative']==1
finance_word_negative = finance_word_dict[is_Negative]

is_Positive =  finance_word_dict['Positive']==1
finance_word_positive = finance_word_dict[is_Positive]

is_Uncertain =  finance_word_dict['Uncertainty']==1
finance_word_uncertain = finance_word_dict[is_Uncertain]

is_Litigious =  finance_word_dict['Litigious']==1
finance_word_litigious = finance_word_dict[is_Litigious]

is_Constraining =  finance_word_dict['Constraining']==1
finance_word_constraining = finance_word_dict[is_Constraining]

is_Interesting =  finance_word_dict['Interesting']==1
finance_word_interesting = finance_word_dict[is_Interesting]

is_Superfluous =  finance_word_dict['Superfluous']==1
finance_word_superfluous = finance_word_dict[is_Superfluous]

In [0]:
%%time
postive_words_count=[]
negative_words_count=[]
polarity_score = []
uncertain_words_count=[]
litigious_words_count=[]
constraining_words_count=[]
interesting_words_count=[]
superfluous_words_count=[]


for word_list in fin_reports3['clean_text_lemmatized']:
       
        print("word list length = ",len(word_list))
        
        pos_word_dict = finance_word_positive['Word'].tolist()
        neg_word_dict = finance_word_negative['Word'].tolist()
        uncertain_word_dict = finance_word_uncertain['Word'].tolist()
        litigious_word_dict = finance_word_litigious['Word'].tolist()
        constraining_word_dict = finance_word_constraining['Word'].tolist()
        interesting_word_dict = finance_word_interesting['Word'].tolist()
        superfluous_word_dict = finance_word_superfluous['Word'].tolist()
        
        pos_words = [item for item in word_list if item in pos_word_dict]
        neg_words = [item for item in word_list if item in neg_word_dict]
        uncertain_words = [item for item in word_list if item in uncertain_word_dict]
        litigious_words = [item for item in word_list if item in litigious_word_dict]
        constraining_words = [item for item in word_list if item in constraining_word_dict]
        interesting_words = [item for item in word_list if item in interesting_word_dict]
        superfluous_words = [item for item in word_list if item in superfluous_word_dict]
        
        pos_len = len(pos_words)
        neg_len = len(neg_words)
        pol_score = (pos_len - neg_len)/(pos_len + neg_len)
        #print("postive words length = ",pos_len)
        #print(list(pos_words))
        #print("negative words length = ",neg_len)
        #print(list(neg_words))
        
        postive_words_count.append(pos_len)
        negative_words_count.append(neg_len)
        polarity_score.append(pol_score)
        uncertain_words_count.append(len(uncertain_words))
        litigious_words_count.append(len(litigious_words))
        constraining_words_count.append(len(constraining_words))
        interesting_words_count.append(len(interesting_words))
        superfluous_words_count.append(len(superfluous_words))
        

fin_reports3['postive_words_count'] = postive_words_count
fin_reports3['negative_words_count'] = negative_words_count
fin_reports3['polarity_score'] = polarity_score
fin_reports3['uncertain_words_count'] = uncertain_words_count
fin_reports3['litigious_words_count'] = litigious_words_count
fin_reports3['constraining_words_count'] = constraining_words_count
fin_reports3['interesting_words_count'] = interesting_words_count
fin_reports3['superfluous_words_count'] = superfluous_words_count


word list length =  99649
word list length =  26447
word list length =  112806
word list length =  25222
word list length =  7981
word list length =  31679
word list length =  7721
word list length =  60364
word list length =  32183
word list length =  21409
word list length =  8309
word list length =  18988
word list length =  44289
word list length =  13185
word list length =  44514
word list length =  21275
word list length =  36324
word list length =  25826
word list length =  34141
word list length =  3146
word list length =  5339
word list length =  35643
word list length =  33061
word list length =  40305
word list length =  6479
word list length =  32061
word list length =  5120
word list length =  29370
word list length =  3039
word list length =  29740
word list length =  31464
word list length =  16589
word list length =  17466
word list length =  9552
word list length =  20362
word list length =  3132
word list length =  20374
word list length =  26169
word list length =  5

In [0]:
fin_reports3.head(2)

,ticker,cik,filing_date,10k_link,long_term_outlook,report_data,filing_year,word_count,clean_text_lemmatized,clean_count,clean_unique_words,clean_unique_word_count,report_string_list,postive_words_count,negative_words_count,polarity_score,uncertain_words_count,litigious_words_count,constraining_words_count,interesting_words_count,superfluous_words_count
0,FE,1031296,2012-02-28,https://www.sec.gov/Archives/edgar/data/103129...,0,k fe x k htm k fe k united states securities ...,2012,161216,"[united, state, security, exchange, commission...",99649,"[scrubber, solid, llpcleveland, inquiry, prepa...",3871,united state security exchange commissionwashi...,912,2589,-0.479006,1275,2556,1301,806,8
1,CL,21665,2015-02-19,https://www.sec.gov/Archives/edgar/data/21665/...,0,k cl x k htm colgate palmolive k cl k united ...,2015,44390,"[colgate, palmolive, united, state, security, ...",26447,"[solid, laundry, inquiry, prepare, indicate, u...",2215,colgate palmolive united state security exchan...,430,822,-0.313099,385,451,227,111,3


In [0]:
# Passing the below columns to build the model
fin_reports4 = fin_reports3[['cik','filing_year','clean_count','clean_unique_word_count','postive_words_count','negative_words_count','polarity_score','uncertain_words_count','litigious_words_count','constraining_words_count','interesting_words_count','superfluous_words_count','long_term_outlook']]

In [0]:
fin_reports4.to_csv('AFter_using_dict_train_1000.csv',index=False)

**Train test split**

In [0]:
from sklearn.model_selection import train_test_split

y=fin_reports4['long_term_outlook']
X=fin_reports4.drop('long_term_outlook',axis=1)
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.20,random_state=120)

**Build a Random forest Classifier**

In [0]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10,max_depth=8)
clf.fit(X=X_train, y=y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
from  sklearn.metrics  import accuracy_score
predicted= clf.predict(X_test)
print(accuracy_score(y_test,predicted))


0.765


In [0]:
from sklearn.metrics import classification_report

print("Classification Report on Test Data")
print(classification_report(y_test,predicted,digits=2))

Classification Report on Test Data
              precision    recall  f1-score   support

           0       0.77      0.19      0.30        54
           1       0.76      0.98      0.86       146

    accuracy                           0.77       200
   macro avg       0.77      0.58      0.58       200
weighted avg       0.77      0.77      0.71       200



**I get a 0.58 f1-score on Validation data**